In [1]:
import cv2
import numpy as np


In [2]:
image_BGR = cv2.imread('../data/cat.jpg')
rows, cols, c = image_BGR.shape
rows, cols, c

(360, 480, 3)

In [3]:
def convert_from_BGR(img_BGR, dst_fmt:str):
    """
      @img_BGR: the result of cv2.imread(file_name)

    Referecens:
      1. https://paaatrick.com/2020-01-26-yuv-pixel-formats/
      2. https://blog.csdn.net/mydear_11000/article/details/50404084
      3. https://www.fourcc.org/yuv.php
    """
    rows, cols, c = img_BGR.shape
    assert (c == 3) and (img_BGR.dtype == np.uint8)
    if dst_fmt == 'RGB':
        return cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)
    elif dst_fmt == 'YUV444':
        """
        Y0 Y1 Y2 Y3 ......     shape: [rows, cols]
        ......
        U0 U1 U2 V3 ......     shape: [rows, cols]
        ......
        V0 V1 V2 V3 ......     shape: [rows, cols]
        ......
        """
        YUV444 = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2YUV)
        y, u, v = YUV444[:, :, 0], YUV444[:, :, 1], YUV444[:, :, 2]
        return np.vstack((y, u, v))
    elif dst_fmt in ['YUV420P', 'YU12', 'IYUV']:
        """
        Y0 Y1 Y2 Y3 ......     shape: [rows, cols]
        ......
        U0 U1 U2 V3 ......     shape: [rows//4, cols]
        ......
        V0 V1 V2 V3 ......     shape: [rows//4, cols]
        ......
        """
        return cv2.cvtColor(image_BGR, cv2.COLOR_BGR2YUV_I420)
    elif dst_fmt == 'YV12':
        """
        Y0 Y1 Y2 Y3 ......     shape: [rows, cols]
        ......
        V1 V1 V2 V3 ......     shape: [rows//4, cols]
        ......
        U0 U1 U2 V3 ......     shape: [rows//4, cols]
        ......
        """
        return cv2.cvtColor(image_BGR, cv2.COLOR_BGR2YUV_YV12)
    elif dst_fmt in ['YUV420SP', 'NV12']:
        """
        Y0 Y1 Y2 Y3 ......     shape: [rows, cols]
        ......
        U0 V0 U1 V1 ......     shape: [rows//2, cols]
        ......
        """
        YUV420P = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2YUV_I420)
        y, u, v = YUV420P[0:rows], YUV420P[rows:-rows//4], YUV420P[-rows//4:]
        uv = np.zeros(shape=[rows//2, cols])
        uv[:, 0::2] = u.reshape(rows//2, -1)
        uv[:, 1::2] = v.reshape(rows//2, -1)
        return np.vstack((y, uv))
    elif dst_fmt == 'YUV422SP':
        """
        Y0 Y1 Y2 Y3 ......     shape: [rows, cols]
        ......
        U0 V0 U1 V1 ......     shape: [rows, cols]
        ......
        """
        YUV = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2YUV)
        y, u, v = YUV[:, :, 0], YUV[:, :, 1], YUV[:, :, 2]
        u = cv2.resize(u, (cols//2, rows), interpolation=cv2.INTER_LINEAR)
        v = cv2.resize(v, (cols//2, rows), interpolation=cv2.INTER_LINEAR)

        # Convert u and v to uint8 with clipping and rounding:
        u = np.round(np.clip(u, 0, 255)).astype(np.uint8)
        v = np.round(np.clip(v, 0, 255)).astype(np.uint8)

        # Interleave u and v:
        uv = np.zeros_like(y)
        uv[:, 0::2] = u
        uv[:, 1::2] = v
        return cv2.merge((y, uv))
    elif dst_fmt == 'YUV422SP_v2':
        YUV = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2YUV)
        y, u, v = YUV[:, :, 0], YUV[:, :, 1], YUV[:, :, 2]

        uv = np.zeros_like(y)
        uv[:, 0::2] = u[:, 0::2]
        uv[:, 1::2] = v[:, 0::2]
        return np.vstack((y, uv))

In [4]:
def save_as_hex(file_name, image):
    """
      @image: astype(np.uint8)
    """
    text_file = open(file_name, "w")
    n = text_file.write(image.reshape(-1).tobytes().hex().upper())
    text_file.close()

In [9]:
image_YUV = convert_from_BGR(image_BGR, 'YUV420SP')
save_as_hex('cat_YUV444.txt', image_YUV)